<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-cleaning" data-toc-modified-id="Data-cleaning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data cleaning</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocessing</a></span><ul class="toc-item"><li><span><a href="#TODOs" data-toc-modified-id="TODOs-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>TODOs</a></span></li></ul></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="#Eligible-population-stats" data-toc-modified-id="Eligible-population-stats-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Eligible population stats</a></span></li></ul></div>

This notebook is from Fall 2022. 
It uses the court data we scraped in October 2021 to model cannabis policy in VA. Note that this is so old I had to change a couple things to get it to work again, so probably worth cross referencing against other more recent policies that use the same dataset and it might warrant some preprocessing updates.

In [7]:
import os
import sys
sys.path.insert(0, os.path.relpath('../')) 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from super_simulation.super_simulation_factory import SuperSimulationFactory
from transition_table import TransitionTable
from shell_compartment import ShellCompartment
from spark_policy import SparkPolicy
from utils.spark_bq_utils import upload_spark_model_inputs
import pandas as pd
from functools import partial

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

# Data cleaning
Tip: you'll probably want to save the preprocessed file to a csv the first time you run this for a new policy and then import that csv from then on out to save time. To do so, uncomment the `df.to_csv()` line at the bottom of
this section, then after you run it move that file to `recidiviz-staging>recidiviz>calculator>modeling>population_projection>state>VA>VA_data`. From then on, you can skip everything in the data cleaning section and just run the file import commented out at the end of the section.

In [8]:
crime_types = [
    'CONSPIRACY', 'DRUG_POSSESSION', 'DRUG_SALE', "DRUG_OTHER", "KIDNAPPING", "MANSLAUGHTER", "LARCENY", "WEAPON",
    "ASSAULT", "ROBBERY", "BURGLARY", "DWI", "RAPE", "SEX_OFFENSE", "ARSON", 'FIRST_DEGREE_MURDER',
    'SECOND_DEGREE_MURDER', 'OTHER_MURDER', 'CAPITAL_MURDER'
]

LOS_BY_CRIME = {
    'CONSPIRACY': 0.899,
    'DRUG_POSSESSION': 0.897,
    'DRUG_SALE': 0.902,
    'DRUG_OTHER': 0.901,
    'KIDNAPPING': 0.913,
    'MANSLAUGHTER': 0.916,
    'LARCENY': 0.907,
    'WEAPON': 0.905,
    'ASSAULT': 0.912,
    'ROBBERY': 0.920,
    'BURGLARY': 0.917,
    'DWI': 0.893,
    'RAPE': 0.906,
    'SEX_OFFENSE': 0.90,
    'ARSON': 0.914,
    'FIRST_DEGREE_MURDER': 0.907,
    'SECOND_DEGREE_MURDER': 0.914,
    'OTHER_MURDER': 0.908,
    'CAPITAL_MURDER': 0.898
}


In [9]:
sdc_data = pd.concat([
    pd.read_csv(
        f'../state/VA/VA_data/VA_circuit_data/circuit_data_{i}.csv',
        index_col=0
    ) for i in range(8)
])

C:\Users\isaac\AppData\Local\Temp\ipykernel_7392\110041657.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(
C:\Users\isaac\AppData\Local\Temp\ipykernel_7392\110041657.py:2: DtypeWarning: Columns (18,24) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(
C:\Users\isaac\AppData\Local\Temp\ipykernel_7392\110041657.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(
C:\Users\isaac\AppData\Local\Temp\ipykernel_7392\110041657.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(
C:\Users\isaac\AppData\Local\Temp\ipykernel_7392\110041657.py:2: DtypeWarning: Columns (18,24) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(
C:\Users\isaac\AppData\Local\Temp\ipykernel_7392\110041657.py:2: DtypeWarning: Columns (1

In [10]:
sdc_data

,Address,Amended Charge,Amended Charge Type,Amended Code Section,Arrest Date,Charge,Charge Type,Class,CodeSection,Commenced by,Concluded By,ConcurrentConsecutive,Costs,Court/DMV Surrender,DOB,Defendant,Disposition Date,Driver Improvement Clinic,Driving Restrictions,Filed,Fine Amount,Fines/Cost Paid,Jail/Penitentiary,Life/Death,Locality,Military,Offense Date,Operator License Suspension Time,Probation Starts,Probation Time,Probation Type,Program Type,Race,Restitution Amount,Restitution Paid,Sentence Suspended,Sentence Time,Sex,Traffic Fatality,VA Alcohol Safety Action,county,time_step,day,month,year,sentence_time,sentence_suspended,person_id,compartment_duration,CONSPIRACY,FIRST_DEGREE_MURDER,SECOND_DEGREE_MURDER,CAPITAL_MURDER,OTHER_MURDER,DRUG_POSSESSION,DRUG_SALE,DRUG_OTHER,ARSON,SEX_OFFENSE,DWI,WEAPON,BURGLARY,ROBBERY,LARCENY,ASSAULT,RAPE,KIDNAPPING,MANSLAUGHTER,crimes,CONSPIRACY_ANY,DRUG_POSSESSION_ANY,DRUG_SALE_ANY,DRUG_OTHER_ANY,KIDNAPPING_ANY,MANSLAUGHTER_ANY,LARCENY_ANY,WEAPON_ANY,ASSAULT_ANY,ROBBERY_ANY,BURGLARY_ANY,DWI_ANY,RAPE_ANY,SEX_OFFENSE_ANY,ARSON_ANY,FIRST_DEGREE_MURDER_ANY,SECOND_DEGREE_MURDER_ANY,OTHER_MURDER_ANY,CAPITAL_MURDER_ANY,total_sentence_months,crime_type
0,LYNCHBURG[.] VA 24506,NaN,NaN,NaN,NaN,RECKLESS DRIVING (SP 95/65 MR),Misdemeanor,NaN,46.2-862,General District Court Appeal,Trial - Judge With Witness,NaN,$88.00,License Was Not Surrendered,03/15/****,A GHAYYADA[.] AMER O,12/11/2002,NaN,Yes,10/15/2002,$500.00,Yes,Jail,NaN,COMMONWEALTH OF VA,NaN,08/23/2002,6 Month(s),NaN,NaN,NaN,NaN,White,NaN,NaN,0 Year(s)0 Month(s)90 Day(s),0 Year(s)0 Month(s)90 Day(s),Male,NaN,NaN,Louisa Circuit Court,-83.638356,11,12,2002,0 Year(s)0 Month(s)90 Day(s),0 Year(s)0 Month(s)90 Day(s),A GHAYYADA[.] AMER O03/15/****LYNCHBURG[.] VA ...,0.000000,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.000000,NaN
1,ROANOKE[.] VA 24017,NaN,NaN,NaN,12/05/2012,DRIVING WHILE INTOXICATED[.] 1ST,Misdemeanor,1,A.18.2-266,General District Court Appeal,Trial - Jury,NaN,$1473.00,License Was Not Surrendered,01/09/****,A'SIM[.] ZAHIR RAKIN; JR,02/17/2012,NaN,Yes,06/15/2011,$1250.00,NaN,Jail,NaN,COMMONWEALTH OF VA,NaN,11/20/2010,12 Month(s),NaN,NaN,NaN,NaN,Black,$0.00,NaN,NaN,7 Day(s),Male,No,Yes,Roanoke City Circuit Court,26.558904,17,2,2012,7 Day(s),0,A'SIM[.] ZAHIR RAKIN; JR01/09/****ROANOKE[.] V...,0.230137,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,0.230137,DWI
2,WILLIAMSBURG[.] VA 23188,NaN,NaN,NaN,03/05/1997,IMPROPER DRIVING,M,O,46.2-869,General District Court Appeal,Trial - Judge With Witness,NaN,$71.00,NaN,11/03/****,AADAHL[.] CHRISTOPHER P.,06/24/1997,NaN,NaN,04/29/1997,$250.00,Yes,NaN,NaN,COMMONWEALTH OF VA,NaN,03/05/1997,NaN,NaN,NaN,NaN,NaN,White,NaN,NaN,NaN,NaN,Male,NaN,NaN,York County/Poquoson Circuit Court,-149.210959,24,6,1997,0,0,AADAHL[.] CHRISTOPHER P.11/03/****WILLIAMSBURG...,0.000000,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.000000,NaN
3,WOODBRIDGE[.] VA 22191,NaN,NaN,NaN,04/21/2015,AP-G POSSESSION OF MARIJUANA,Misdemeanor,U,18.2-250.1,General District Court Appeal,Trial - Judge With Witness,NaN,$477.50,License Was Not Surrendered,06/19/****,AAGESEN[.] KEVIN MICHAEL,10/01/2015,NaN,NaN,07/09/2015,$350.00,NaN,NaN,NaN,COMMONWEALTH OF VA,NaN,04/21/2015,6 Month(s),NaN,NaN,NaN,NaN,Asian Or Pacific Islander,$0.00,NaN,NaN,NaN,Male,No,NaN,Prince William Circuit Court,70.032877,1,10,2015,0,0,AAGESEN[.] KEVIN MICHAEL06/19/****WOODBRIDGE[....,0.000000,False,False,False,False,False,True,

In [11]:
# Check sentencing data to determine age of offenders when sentenced
raw_sent = pd.concat([
    pd.read_csv(f'../state/VA/VA_data/sentencing_data/sentencing_{i}.csv', encoding = "ISO-8859-1") for i in range(2019,2022)
])

under_25 = 0
total = 0

for i, row in raw_sent.iterrows():
    sentence_year = row['Calendar Year of Sentencing']
    birth_year = row['Offender Birth Year']
    
    sentencing_age = sentence_year - birth_year
        
    if sentencing_age <= 25:
        under_25 += 1
    
    total += 1
        
# Proportion of inmates incarcerated before 25
INCARCERATED_BEFORE_25 = under_25/total

C:\Users\isaac\AppData\Local\Temp\ipykernel_7392\3001792234.py:3: DtypeWarning: Columns (40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(f'../state/VA/VA_data/sentencing_data/sentencing_{i}.csv', encoding = "ISO-8859-1") for i in range(2019,2022)
C:\Users\isaac\AppData\Local\Temp\ipykernel_7392\3001792234.py:3: DtypeWarning: Columns (40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(f'../state/VA/VA_data/sentencing_data/sentencing_{i}.csv', encoding = "ISO-8859-1") for i in range(2019,2022)
C:\Users\isaac\AppData\Local\Temp\ipykernel_7392\3001792234.py:3: DtypeWarning: Columns (34,35,37,38,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(f'../state/VA/VA_data/sentencing_data/sentencing_{i}.csv', encoding = "ISO-8859-1") for i in range(2019,2022)


In [12]:
sdc_data

,Address,Amended Charge,Amended Charge Type,Amended Code Section,Arrest Date,Charge,Charge Type,Class,CodeSection,Commenced by,Concluded By,ConcurrentConsecutive,Costs,Court/DMV Surrender,DOB,Defendant,Disposition Date,Driver Improvement Clinic,Driving Restrictions,Filed,Fine Amount,Fines/Cost Paid,Jail/Penitentiary,Life/Death,Locality,Military,Offense Date,Operator License Suspension Time,Probation Starts,Probation Time,Probation Type,Program Type,Race,Restitution Amount,Restitution Paid,Sentence Suspended,Sentence Time,Sex,Traffic Fatality,VA Alcohol Safety Action,county,time_step,day,month,year,sentence_time,sentence_suspended,person_id,compartment_duration,CONSPIRACY,FIRST_DEGREE_MURDER,SECOND_DEGREE_MURDER,CAPITAL_MURDER,OTHER_MURDER,DRUG_POSSESSION,DRUG_SALE,DRUG_OTHER,ARSON,SEX_OFFENSE,DWI,WEAPON,BURGLARY,ROBBERY,LARCENY,ASSAULT,RAPE,KIDNAPPING,MANSLAUGHTER,crimes,CONSPIRACY_ANY,DRUG_POSSESSION_ANY,DRUG_SALE_ANY,DRUG_OTHER_ANY,KIDNAPPING_ANY,MANSLAUGHTER_ANY,LARCENY_ANY,WEAPON_ANY,ASSAULT_ANY,ROBBERY_ANY,BURGLARY_ANY,DWI_ANY,RAPE_ANY,SEX_OFFENSE_ANY,ARSON_ANY,FIRST_DEGREE_MURDER_ANY,SECOND_DEGREE_MURDER_ANY,OTHER_MURDER_ANY,CAPITAL_MURDER_ANY,total_sentence_months,crime_type
0,LYNCHBURG[.] VA 24506,NaN,NaN,NaN,NaN,RECKLESS DRIVING (SP 95/65 MR),Misdemeanor,NaN,46.2-862,General District Court Appeal,Trial - Judge With Witness,NaN,$88.00,License Was Not Surrendered,03/15/****,A GHAYYADA[.] AMER O,12/11/2002,NaN,Yes,10/15/2002,$500.00,Yes,Jail,NaN,COMMONWEALTH OF VA,NaN,08/23/2002,6 Month(s),NaN,NaN,NaN,NaN,White,NaN,NaN,0 Year(s)0 Month(s)90 Day(s),0 Year(s)0 Month(s)90 Day(s),Male,NaN,NaN,Louisa Circuit Court,-83.638356,11,12,2002,0 Year(s)0 Month(s)90 Day(s),0 Year(s)0 Month(s)90 Day(s),A GHAYYADA[.] AMER O03/15/****LYNCHBURG[.] VA ...,0.000000,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.000000,NaN
1,ROANOKE[.] VA 24017,NaN,NaN,NaN,12/05/2012,DRIVING WHILE INTOXICATED[.] 1ST,Misdemeanor,1,A.18.2-266,General District Court Appeal,Trial - Jury,NaN,$1473.00,License Was Not Surrendered,01/09/****,A'SIM[.] ZAHIR RAKIN; JR,02/17/2012,NaN,Yes,06/15/2011,$1250.00,NaN,Jail,NaN,COMMONWEALTH OF VA,NaN,11/20/2010,12 Month(s),NaN,NaN,NaN,NaN,Black,$0.00,NaN,NaN,7 Day(s),Male,No,Yes,Roanoke City Circuit Court,26.558904,17,2,2012,7 Day(s),0,A'SIM[.] ZAHIR RAKIN; JR01/09/****ROANOKE[.] V...,0.230137,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,0.230137,DWI
2,WILLIAMSBURG[.] VA 23188,NaN,NaN,NaN,03/05/1997,IMPROPER DRIVING,M,O,46.2-869,General District Court Appeal,Trial - Judge With Witness,NaN,$71.00,NaN,11/03/****,AADAHL[.] CHRISTOPHER P.,06/24/1997,NaN,NaN,04/29/1997,$250.00,Yes,NaN,NaN,COMMONWEALTH OF VA,NaN,03/05/1997,NaN,NaN,NaN,NaN,NaN,White,NaN,NaN,NaN,NaN,Male,NaN,NaN,York County/Poquoson Circuit Court,-149.210959,24,6,1997,0,0,AADAHL[.] CHRISTOPHER P.11/03/****WILLIAMSBURG...,0.000000,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.000000,NaN
3,WOODBRIDGE[.] VA 22191,NaN,NaN,NaN,04/21/2015,AP-G POSSESSION OF MARIJUANA,Misdemeanor,U,18.2-250.1,General District Court Appeal,Trial - Judge With Witness,NaN,$477.50,License Was Not Surrendered,06/19/****,AAGESEN[.] KEVIN MICHAEL,10/01/2015,NaN,NaN,07/09/2015,$350.00,NaN,NaN,NaN,COMMONWEALTH OF VA,NaN,04/21/2015,6 Month(s),NaN,NaN,NaN,NaN,Asian Or Pacific Islander,$0.00,NaN,NaN,NaN,Male,No,NaN,Prince William Circuit Court,70.032877,1,10,2015,0,0,AAGESEN[.] KEVIN MICHAEL06/19/****WOODBRIDGE[....,0.000000,False,False,False,False,False,True,

In [13]:
# This is to make it easier to label data based on violation codes
codes = sdc_data['CodeSection'].unique().astype(str)
charges = sdc_data['Charge'].unique().astype(str)

def containing(a, substr):
    b = []
    for el in a:
        if substr.lower() in el.lower():
            b.append(el)
    return b

containing(codes, "18.2-374.1")

['18.2-374.1',
 '18.2-374.1 B1)',
 '18.2-374.1B2',
 '18.2-374.1-B(2)',
 'B.18.2-374.1',
 '18.2-374.1(B)',
 '18.2-374.1(B[.]2)',
 '18.2-374.1(B)(3',
 '18.2-374.1B3',
 '18.2-374.1(B)4',
 '18.2-374.1(B[.]3)',
 '18.2-374.1(B)(4',
 '18.2-374.1B1&.2',
 '18.2-374.1B(1)',
 '18.2-374.1B(2)',
 '18.2-374.1 B(1)',
 '18.2-374.1(B)2',
 '18.2-374.1A',
 '18.2-374.1B4',
 '18.2-374.1(B[.]1)',
 '18.2-374.1C2',
 '18.2-374.1 B. 2',
 '18.2-374.1(B)(2',
 '18.2-374.1(B)(1',
 '18.2-374.1(B2)',
 '18.2-374.1.1',
 '18.2-374.1(C)',
 'C.18.2-374.1',
 'A.18.2-374.1',
 '18.2-374.1 B.1']

In [14]:
'''
Modeling Assumptions made here:
 - Child-adult distinctions are not made unless under separate codes. Codes are treated under highest severity (ex. in alternative A, 18.2-67.1 committed against an adult is treated as 18.2-67.1 committed against a child)
 - All offenses are assumed repeat where those rules apply (ex. in alternative A, 18.2.30 is always in the 30-year tier)
 - Subsection-level distinctions are not made. If a tier includes some subsection of a code, the whole code is included
 - Inchoate offenses are not factored in
'''
# Note: We can't see repeat offenses in the data as-is, so all rules applying to repeat offenses will be applied to all offense of that type
## This means our model of the policy will generally let people out later, so our estimates will be more conservative
# This also does not include any of the inchoate conviction clauses
# 

policy_scenarios = {
    'original': [
        [],
        [],
        ['F1', '18.2-31', '18.2-30', '18.2-40', '18.2-45', '18.2-46.5', '18.2-46.6', '18.2-61', '18.2-67.1', '18.2-67.2', '18.2-67.3', '18.2-63', '18.2-64.1', '18.2-64.2', '18.2-274.1', '18.2-355', '18.2-371.1', '18.2-481'],
        ['18.2-32', '18.2-41', '18.2-51', '18.2-51.1', '18.2-51.2', '18.2-61', '18.2-67.1', '18.2-67.2', '18.2-67.3', '18.2-63', '18.2-64.1', '18.2-64.2', '18.2.90', '18.2-77', '18.2-79', '18.2-80', '18.2-79', '18.2-80', '18.2-79', '18.2-374.1C', '18.2-374.1(C)'],
        ['F*']
    ],
    'a': [
        [],
        ['F1', '18.2.31', '18.2.30'],
        ['18.2-40', '18.2-45', '18.2-46.5', '18.2-46.6', '18.2-46.6', '18.2-61', '18.2-67.1', '18.2-67.2', '18.2-67.3', '18.2-63', '18.2-64.1', '18.2-64.2', '18.2-64.2', '18.2-274.1', '18.2-274.1', '18.2-355', '18.2-371.1', '18.2-481'],
        ['18.2-32', '18.2-41', '18.2-51', '18.2-51.1', '18.2-51.2', '18.2-61', '18.2-90', '18.2-77', '18.2-79', '18.2-80', '18.2-374.1'],
        ['F*']
    ],
    'b': [
        ['18.2-31', '18.2-30'],
        [],
        ['18.2-61', '18.2-67.1', '18.2-67.2', '18.2-67.3', '18.2-63', '18.2-64.1', '18.2-64.2', '18.2-46.6', '18.2-274.1', '18.2-40', '18.2-45', '18.2-481', '18.2-371.1'],
        [],
        ['F*']
    ],
    'c': [
        ['18.2.31', '18.2-30'],
        [],
        ['18.2-61', '18.2-67.1', '18.2-67.2', '18.2-67.3', '18.2-63', '18.2-64.1', '18.2-64.2', '18.2-46.6', '18.2-274.1', '18.2-40', '18.2-45', '18.2-481', '18.2-371.1'],
        ['18.2-32', '18.2-61', '18.2-67.1', '18.2-67.2', '18.2-67.3', '18.2-63', '18.2-64.1', '18.2-64.2', '18.2-51', '18.2-77', '18.2-79', '18.2-80', '18.2-41', '18.2-51', '18.2-51.1', '18.2-79', '18.2-374.1'],
        ['F*']
    ]
}

# Here you can change the version of the bill we're using
scenario = 'original'

In [15]:
sdc_data['Charge Type'].unique()

array(['Misdemeanor', 'M', 'F', 'Felony', 'Infraction', 'Civil', nan,
       'Other (Animal Violations[.] Bond Appeals)', 'I'], dtype=object)

In [16]:
# Add tier column
sdc_data["tier"] = np.nan

In [17]:
sdc_data['Charge Type'].unique()

array(['Misdemeanor', 'M', 'F', 'Felony', 'Infraction', 'Civil', nan,
       'Other (Animal Violations[.] Bond Appeals)', 'I'], dtype=object)

In [18]:
# Assign tiers for a given policy scenario
for tier, tier_codes in enumerate(policy_scenarios[scenario]):
    for code in tier_codes:
        conditions = None
        # Class-1 felonies
        if code == 'F1':
            conditions = (sdc_data['Class'] == '1') & (sdc_data['Charge Type'].isin(['F', 'Felony']))
        # All other felonies
        if code == 'F*':
            conditions = sdc_data['Charge Type'].isin(['F', 'Felony'])
        # Actual codes
        else:
            # This is for catching minor abberations in how codes are written out
            possible_codes = containing(codes, code) + containing(codes, code.replace('-','.'))
            conditions = sdc_data['CodeSection'].isin(possible_codes)   

        sdc_data.loc[conditions & sdc_data['tier'].isna(), 'tier'] = tier

In [19]:
sdc_data.loc[sdc_data['tier'].isna(), 'tier'] = 0 

In [20]:
sdc_data[sdc_data['Charge'] == 'MURDER']['CodeSection']

490        Z.18.2-32
1092       A.18.2-32
1706         18.2-32
1876       A.18.2-32
2872         18.2-32
             ...    
1069348      18.2-32
1069663      18.2-32
1070613      18.2-33
1070928      18.2-33
1071511      18.2-32
Name: CodeSection, Length: 2106, dtype: object

# Attempt w/o Spark

In [21]:
# Create cleaner, more usable data set

# We're currently missing a bunch of data that doesn't fall into any category, so we need to scale up counts 
# by the ratio of actual total data to the data we're catching
scale_factor = len(sdc_data) / len(sdc_data[sdc_data.crime_type.notnull()])
sorted_data = sdc_data[sdc_data.crime_type.notnull()]

# Calculate LOS using our percentage served data
sorted_data['LOS'] = sorted_data.apply(lambda x: LOS_BY_CRIME[x.crime_type] * x.total_sentence_months, axis=1)

# Drop sentences that will round to 0
sorted_data = sorted_data[sorted_data.LOS > 0.5]

# Calculate time_step column
sorted_data['time_step'] = (sorted_data.year - 2019) * 12 + sorted_data.month

# get relevant info. don't need DOB, but can add later if we want to do that analysis
sorted_data = sorted_data[['time_step', 'tier', 'LOS']].copy()
sorted_data['compartment_duration'] = sorted_data['LOS'].round()
sorted_data = sorted_data.drop('LOS', axis=1)
sorted_data['weight'] = scale_factor  # scale weights to handle missing data [MAYBE CHECK]

# keep only people admitted before 33 [MAYBE CHECK]
sorted_data = sorted_data[(sorted_data['time_step'] <= 33)]  # IS IT OK TO GET RID OF -108 CONDITION? WAS "sorted_data['time_step'] > -108)&"

sorted_data.time_step = sorted_data.time_step.astype(int)

C:\Users\isaac\AppData\Local\Temp\ipykernel_7392\1313892748.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_data['LOS'] = sorted_data.apply(lambda x: LOS_BY_CRIME[x.crime_type] * x.total_sentence_months, axis=1)


In [22]:
sorted_data.head()

,time_step,tier,compartment_duration,weight
14,30,4.0,4.0,1.665042
22,-177,4.0,39.0,1.665042
32,-81,2.0,194.0,1.665042
41,-217,4.0,11.0,1.665042
42,-47,4.0,8.0,1.665042


In [23]:
# Calculate people released at each time step WITHOUT policy. Note we don't care about incoming people -- none are affected
START_STEP = 72 # start of 2025 fiscal year

no_policy_data = sorted_data.copy()

no_policy_data['release_month'] = no_policy_data['time_step'] + no_policy_data['compartment_duration']  # month of release
no_policy_data['release_year'] = (no_policy_data['release_month'] - START_STEP) // 12  # years between policy start (48) and released
no_policy_nums_released = no_policy_data.groupby(['release_year'], as_index=False).sum()

# get number released years 0 though 4
no_policy_yearly_releases = np.array([no_policy_nums_released.loc[no_policy_nums_released['release_year'] == i, 'weight'].values[0] for i in range(5)])
no_policy_total_releases = no_policy_yearly_releases.copy()
for i in range(1, 5):  # get cumulative releases at each time step
    no_policy_total_releases[i] = no_policy_total_releases[i] + no_policy_total_releases[i - 1]

print(no_policy_yearly_releases)
print(no_policy_total_releases)

[1923.12402889 1235.46149735  835.85130953  752.59918706  551.1290507 ]
[1923.12402889 3158.58552623 3994.43683576 4747.03602282 5298.16507352]


In [24]:
def make_cumulative(array):
    new_array = array.copy()
    for i in range(1, len(new_array)):
        new_array[i] = new_array[i] + new_array[i - 1]
    return new_array

In [25]:
import pandas as pd

df = pd.DataFrame({'a': [1]})
df2 = df[df['a'] == 1]
df2['a'] *= 2

df

,a
0,1


In [27]:
# Calculate people released at each time step WITH policy. Note we don't care about incoming people -- none are affected

# This estimates resentencing frequency/outcomes using the parole set-off system used in Texas
# originally found in https://www.secondlooktexas.org/action
GRANT_RATE = 3 / (32 + 3)  

# We don't have data on the distribution of set-off times after each hearing, so doing every other year
SETOFF_TIME = 24

# Months of sentence that must be served to be eligible for Second Look. Tier 0 never eligible
TIER_SENTENCE_LENGTHS = np.array([999999, 30 * 12, 25 * 12, 20 * 12, 15 * 12])


policy_data = sorted_data.copy()

policy_data['release_month'] = policy_data['time_step'] + policy_data['compartment_duration']  # month of release
policy_data['tier_sentence_lengths'] = TIER_SENTENCE_LENGTHS[policy_data['tier'].astype('int64')]  # time when eligible for policy
policy_data['next_chance_to_apply'] = policy_data['time_step'] + policy_data['tier_sentence_lengths']  # next chance to apply (eg after rejection)


additional_releases = np.zeros(5)
policy_yearly_releases = np.zeros(5)
life_years_timed = np.zeros(5)  # life years saved, at time of reference (eg if 3 years early, 1 first year, 2 second, 3 third)
life_years_immediate = np.zeros(5)  # life years saved, applied immediately (eg if 3 years early, 3 first year, 3 second, 3 third)

for year in range(5):
    time = START_STEP + 12 * year  # start month, plus months of years so far, shifted to start in July 2024
    # Select people who can apply before end of year but not set to be released yet [MAYBE CHECK]
    filter = (time + 11 >= policy_data['next_chance_to_apply']) & (time + 11 < policy_data['release_month'])
    eligible_data = policy_data.loc[filter]
    print("Number of eligible applicants in the " + str(year) + " year after passing:", eligible_data['weight'].sum())  # Number of people eligible to apply each year
    additional_releases[year] += GRANT_RATE * eligible_data['weight'].sum()  # release GRANT_RATE proportion of eligible people
    
    early_release_months = eligible_data['release_month'] - (time + 11)  #  in case released at end of year, underestimate for years [MAYBE CHECK]
    # find years saved over time [MAYBE CHECK]
    years_and_weights = pd.DataFrame([early_release_months // 12, eligible_data['weight']])  # df with only years saved and weights
    years_and_weights = years_and_weights.rename({'release_month': 'years_saved'}).transpose()
    years_and_weights = years_and_weights.groupby(['years_saved'], as_index=False).sum()  # sum weights for each year
    for i in range(year, 5):  # add a year saved for each person who would have been in prison in each year
        life_years_timed[i] += years_and_weights.loc[years_and_weights['years_saved'] > i, 'weight'].sum() * GRANT_RATE
    # add total time saved for each person released [MAYBE CHECK]
    for i in range(year, 5):
        life_years_immediate[i] += GRANT_RATE * (eligible_data['weight'] * early_release_months).sum() / 12
    
    policy_data.loc[filter, 'weight'] *= 1 - GRANT_RATE  # take released people out of set
    policy_data.loc[filter, 'next_chance_to_apply'] = time + SETOFF_TIME 

    # release people set to be released normally by end of year [MAYBE CHECK]
    policy_yearly_releases[year] = policy_data.loc[(time + 11 >= policy_data['release_month'])
            & (time - 1 < policy_data['release_month']), 'weight'].sum() + additional_releases[year]


policy_total_releases = make_cumulative(policy_yearly_releases)
additional_releases_cumulative = make_cumulative(additional_releases)
life_years_timed_cumulative = make_cumulative(life_years_timed)
life_years_immediate_cumulative = make_cumulative(life_years_immediate)

print("Additional releases each year:", additional_releases)
print("Additional releases, cumulative:", additional_releases_cumulative)
print("Yearly releases:", policy_yearly_releases)
print("Total releases:", policy_total_releases)
print("Life years, timed: ", life_years_timed)
print("Life years, immediate:", life_years_immediate)
print("Life years, timed, cumulative: ", life_years_timed_cumulative)
print("Life years, immediate, cumulative: ", life_years_immediate_cumulative)

Number of eligible applicants in the 0 year after passing: 1623.4163880222254
Number of eligible applicants in the 1 year after passing: 1564.9971843741291
Number of eligible applicants in the 2 year after passing: 1490.770271595568
Number of eligible applicants in the 3 year after passing: 1430.4949194015933
Number of eligible applicants in the 4 year after passing: 1382.6920763404066
Additional releases each year: [139.14997612 134.1426158  127.78030899 122.61385023 118.51646369]
Additional releases, cumulative: [139.14997612 273.29259192 401.07290091 523.68675115 642.20321483]
Yearly releases: [2062.274005   1360.470166    942.01189183  844.33901438  638.1256109 ]
Total releases: [2062.274005   3422.744171   4364.75606283 5209.09507721 5847.22068811]
Life years, timed:  [132.01407991 234.20011365 315.68750471 387.45674881 442.73935412]
Life years, immediate: [ 2237.73379967  4362.58553679  6415.21994601  8410.10975466
 10275.10108007]
Life years, timed, cumulative:  [ 132.01407991  

In [20]:
# find costs avoided by releasing people
COST_PER_YEAR = 12644.86  # seen in https://vadoc.virginia.gov/media/1813/vadoc-financial-annual-mis-report-2022.pdf

# find valued for timed years
cost_avoided_timed_yearly = life_years_timed.copy() * COST_PER_YEAR
cost_avoided_timed_cumulative = make_cumulative(cost_avoided_timed_yearly)

# find valued for immediate years
cost_avoided_immediate_yearly = life_years_immediate.copy() * COST_PER_YEAR
cost_avoided_immediate_cumulative = make_cumulative(cost_avoided_immediate_yearly)

print("Cost avoided each year, timed:", cost_avoided_timed_yearly)
print("Cumulative cost avoided, timed:", cost_avoided_timed_cumulative)
print("Cost avoided each year, immediate:", cost_avoided_immediate_yearly)
print("Cumulative cost avoided, immediate:", cost_avoided_immediate_cumulative)

Cost avoided each year, timed: [1669299.55842885 3080534.42836546 4316718.42568845 5493349.03335939
 6476882.28670395]
Cumulative cost avoided, timed: [ 1669299.55842885  4749833.98679432  9066552.41248277 14559901.44584216
 21036783.73254611]
Cost avoided each year, immediate: [2.82958306e+07 5.74427032e+07 8.77040965e+07 1.19074296e+08
 1.50479385e+08]
Cumulative cost avoided, immediate: [2.82958306e+07 8.57385338e+07 1.73442630e+08 2.92516926e+08
 4.42996311e+08]


In [21]:
# find labor boost by releasing people
YEARLY_LABOR_BOOST = 18220.8 * 12 / 11  # 18220 calculated a year ago, and wage has since increased from 11 to 12
UNEMPLOYMENT_RATE = 0.27  # found here, nationwide stathttps://www.prisonpolicy.org/reports/outofwork.html


# find valued for timed years
labor_timed_yearly = life_years_timed.copy() * YEARLY_LABOR_BOOST * (1 - UNEMPLOYMENT_RATE)
labor_timed_cumulative = make_cumulative(labor_timed_yearly)

# find valued for immediate years
labor_immediate_yearly = life_years_immediate.copy() * YEARLY_LABOR_BOOST * (1 - UNEMPLOYMENT_RATE)
labor_immediate_cumulative = make_cumulative(labor_immediate_yearly)

print("Labor boost each year, timed:", labor_timed_yearly)
print("Cumulative labor boost, timed:", labor_timed_cumulative)
print("Labor boost each year, immediate:", labor_immediate_yearly)
print("Cumulative labor boost, immediate:", labor_immediate_cumulative)

Labor boost each year, timed: [1915574.80080926 3535012.0918718  4953572.89030893 6303794.26341989
 7432430.22713994]
Cumulative labor boost, timed: [ 1915574.80080926  5450586.89268106 10404159.78298999 16707954.04640988
 24140384.27354982]
Labor boost each year, immediate: [3.24703735e+07 6.59173449e+07 1.00643265e+08 1.36641575e+08
 1.72679922e+08]
Cumulative labor boost, immediate: [3.24703735e+07 9.83877184e+07 1.99030983e+08 3.35672558e+08
 5.08352479e+08]
